In [ ]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
model = model.eval()


In [ ]:
def generate_response(prompt: str, max_new_tokens: int = 128, temperature: float = 0.001) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,

            do_sample = True,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)
     

In [ ]:
def format_prompt(prompt, history='', Background=''):
    return f"""<s>[INST] <<SYS>>
{{ 
    {Background}
    {history}
}}
<</SYS>>
[/INST]  </s>
<s>[INST] {{ {prompt} }} [/INST]"""

In [ ]:
# Write the examples / evidences / history here
history = ''


# Write the main prompt here
prompt = ''

In [ ]:
answer = generate_response(format_prompt(prompt, history))
print(answer)